In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils

In [2]:
events = utils.get_events()

In [3]:
events.head(1)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,month,day,hora
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0


In [4]:
#Busco usuarios que compraron
conversions = events.loc[events.event == 'conversion']
users_with_conversions = conversions.person.unique()
print("Hay {} usuarios que hicieron al menos una compra".format(len(users_with_conversions)))

Hay 4293 usuarios que hicieron al menos una compra


In [5]:
#agrego columna "compro" para los users de users_with_conversions
events['compro'] = np.where(events.person.isin(users_with_conversions),True,False)

In [6]:
eventos_de_usuarios_que_compraron = events.loc[events.compro]
tabla_compraron = utils.get_features_iniciales(eventos_de_usuarios_que_compraron,'event')
tabla_compraron.describe()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
count,4293.000000,4293.000000,4293.000000,4293.000000,4293.000000,4293.000000,4293.000000,4293.000000,4293.000000,4293.000000,4293.000000
mean,10.046122,15.599581,4.219427,1.651759,12.310040,0.089215,5.792686,8.571395,1.473329,76.337992,14.163522
std,18.329101,40.224523,5.691987,2.528974,20.755131,0.530705,10.415970,23.744099,3.651979,140.947131,19.890563
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,1.000000,1.000000,2.000000,0.000000,1.000000,0.000000,0.000000,10.000000,3.000000
50%,4.000000,4.000000,3.000000,1.000000,6.000000,0.000000,3.000000,1.000000,0.000000,31.000000,7.000000
75%,12.000000,15.000000,5.000000,2.000000,14.000000,0.000000,7.000000,8.000000,1.000000,84.000000,17.000000
max,424.000000,1346.000000,197.000000,129.000000,267.000000,11.000000,245.000000,739.000000,69.000000,2355.000000,295.000000


In [7]:
eventos_de_usuarios_que_no_compraron = events.loc[events.compro == False]
tabla_no_compraron = utils.get_features_iniciales(eventos_de_usuarios_que_no_compraron,'event')
tabla_no_compraron.describe()

event,ad campaign hit,brand listing,checkout,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
count,34536.000000,34536.00000,34536.000000,34536.000000,34536.000000,34536.000000,34536.000000,34536.000000,34536.000000,34536.000000
mean,4.292912,4.32427,1.366719,3.107743,0.017373,2.360957,2.716557,0.141186,26.650596,4.148280
std,9.287517,20.06129,1.837912,7.444516,0.319329,6.197998,10.637783,0.800528,66.415906,7.735383
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000
50%,2.000000,0.00000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,8.000000,2.000000
75%,5.000000,3.00000,1.000000,3.000000,0.000000,3.000000,1.000000,0.000000,25.000000,4.000000
max,523.000000,1891.00000,101.000000,470.000000,38.000000,762.000000,634.000000,53.000000,2233.000000,294.000000


In [8]:
eventos_de_usuarios_que_compraron[['person','timestamp','event']].sort_values('person').to_csv('mateo.csv')

### feature_top_modelos
La idea es asignar True/False si el modelo más frecuente relacionado al user es uno de los tops

In [91]:
por_modelo = events.groupby(['model','event']).agg({'event':'count'}).unstack(fill_value=0)
por_modelo.columns = por_modelo.columns.droplevel(level=0).astype(object)
q=0.9
porcentaje = round((1-q)*100)
limite_viewed = por_modelo['viewed product'].quantile(q)
limite_conversion = por_modelo.conversion.quantile(q)
limite_checkout = por_modelo.checkout.quantile(q)
print("Para estar en el top {}% de checkouts un modelo requiere tener {} eventos checkout".format(porcentaje,round(limite_checkout)))
print("Para estar en el top {}% de conversions un modelo requiere tener {} eventos conversion".format(porcentaje,round(limite_conversion)))
print("Para estar en el top {}% de viewed un modelo requiere tener {} eventos viewed".format(porcentaje,round(limite_viewed)))
top_viewed = por_modelo['viewed product'] > limite_viewed
top_checkout = por_modelo.checkout > limite_checkout
top_conversion = por_modelo.conversion > limite_conversion
top_modelos = por_modelo.loc[top_viewed | top_checkout | top_conversion]
top_modelos = set(top_modelos.index)
print("Modelos Top: {}".format(top_modelos))

Para estar en el top 10% de checkouts un modelo requiere tener 798 eventos checkout
Para estar en el top 10% de conversions un modelo requiere tener 94 eventos conversion
Para estar en el top 10% de viewed un modelo requiere tener 14002 eventos viewed
Modelos Top: {'Samsung Galaxy A7 2017', 'Samsung Galaxy S5', 'Motorola Moto G4 Plus', 'iPhone 5', 'iPhone 4S', 'Samsung Galaxy S8 Plus', 'Samsung Galaxy J2 Prime TV', 'iPhone 6 Plus', 'iPhone 6', 'Samsung Galaxy S6 Flat', 'iPhone 7 Plus', 'iPhone 5s', 'Samsung Galaxy Gran Prime Duos TV', 'iPhone 6S Plus', 'Samsung Galaxy S7 Edge', 'Motorola Moto G2 3G Dual', 'Samsung Galaxy J5', 'Samsung Galaxy S8', 'Samsung Galaxy S7', 'iPhone 7', 'Samsung Galaxy J7 Prime', 'Samsung Galaxy S6 Edge', 'iPhone 6S', 'Samsung Galaxy J7', 'Samsung Galaxy A5 2017', 'iPhone SE', 'iPhone 5c'}


In [92]:
frecuencia_modelos_por_persona = events.groupby(['person']).model.value_counts().unstack(fill_value=0)
frecuencia_modelos_por_persona['most_frequent_model'] = frecuencia_modelos_por_persona.idxmax(axis=1)
features = pd.DataFrame(data=frecuencia_modelos_por_persona.most_frequent_model,columns=['most_frequent_model'])
features['is_most_frequent_model_top'] = features.most_frequent_model.isin(top_modelos)

In [83]:
features.head()

,most_frequent_model,is_most_frequent_model_top,person,convirtio,top_y_convirtio,no_top_y_no_convirtio
person,,,,,,
0008ed71,LG G4 H818P,False,0008ed71,False,False,True
00091926,iPhone 6S,True,00091926,False,False,False
00091a7a,iPhone 6,True,00091a7a,False,False,False
000ba417,Samsung Galaxy A3 Duos,False,000ba417,True,False,False
000c79fe,iPhone 7,True,000c79fe,False,False,False


In [93]:
#trato de verificar validez del supuesto:
features['person']= features.index
features['convirtio'] = features.person.isin(conversions.person)
features['top_y_no_convirtio'] = features.is_most_frequent_model_top & ~features.convirtio
features['no_top_y_no_convirtio'] = (~features.is_most_frequent_model_top) & (~features.convirtio)
features.no_top_y_no_convirtio.describe()

count     38256
unique        2
top       False
freq      30470
Name: no_top_y_no_convirtio, dtype: object

In [94]:
features.top_y_no_convirtio.describe()

count     38256
unique        2
top        True
freq      26177
Name: top_y_no_convirtio, dtype: object